In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Web Dash
from flask import Flask
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
import datetime

from abides_core import abides
from abides_core.utils import parse_logs_df, ns_date, str_to_ns, fmt_ts
from abides_markets.configs import rmsc05VAR

In [ ]:
config = rmsc05VAR.build_config(
    end_time="10:00:00",
    seed=1337,
)

config.keys()
end_state = abides.run(config)

logs_df = parse_logs_df( end_state )

In [ ]:
"""
    Get the Order book from the Exchange 0.
"""
Ex_0_order_book = end_state["agents"][0].order_books["ABM"]
ex_0_name = Ex_0_order_book.owner.name
ex_0_ob_imbalance = Ex_0_order_book.get_imbalance()
Ex_0_L1 = Ex_0_order_book.get_L1_snapshots()
Ex_0_L2 = Ex_0_order_book.get_L2_snapshots(nlevels=10)


In [ ]:
times = [ t - ns_date(t) for t in Ex_0_L2["times"] ]
times = list(map(int, times))


bid_prices = Ex_0_L2["bids"][:,0,0]
ask_prices = Ex_0_L2["asks"][:,0,0]
bid_prices = np.divide(bid_prices, 100)
ask_prices = np.divide(ask_prices, 100)


Ex_0_fig = go.Figure(layout_yaxis_range =[998, 1006])
Ex_0_fig.add_trace(go.Scatter(x=times, y=bid_prices, mode='markers', marker_size=3, name='best_bids'))
Ex_0_fig.add_trace(go.Scatter(x=times, y=ask_prices, mode='markers', marker_size=3, name='best_asks'))
Ex_0_fig.update_layout(title='Best Bid / Ask Prices Exchange 0', xaxis_title='Time', yaxis_title='Price')


In [ ]:

"""
    Price / Timeseries Plotting
"""
Ex0_best_bids = pd.DataFrame(Ex_0_L1["best_bids"],columns=["time","price","qty"])
Ex0_best_asks = pd.DataFrame(Ex_0_L1["best_asks"],columns=["time","price","qty"])
# divide all prices by 100 
Ex0_best_bids['price'] = Ex0_best_bids['price'].div(100)
Ex0_best_asks['price'] = Ex0_best_asks['price'].div(100)
# remove all nan values
# Ex0_best_bids = Ex0_best_bids.dropna()
# Ex0_best_asks = Ex0_best_asks.dropna()
# remove all time duplicates
# Ex0_best_bids = Ex0_best_bids.drop_duplicates(subset=['time'])
# Ex0_best_asks = Ex0_best_asks.drop_duplicates(subset=['time'])


Ex_0_fig = go.Figure()
Ex_0_fig.add_trace(go.Scatter(x=Ex0_best_bids["time"], y=Ex0_best_bids["price"], mode='markers', marker_size=3, name='best_bids'))
Ex_0_fig.add_trace(go.Scatter(x=Ex0_best_bids["time"], y=Ex0_best_asks["price"], mode='markers', marker_size=3, name='best_asks'))
Ex_0_fig.update_layout(title='Order book of Exchange 0', xaxis_title='Time', yaxis_title='Price')

In [ ]:
executed_orders =  logs_df[logs_df.EventType=="ORDER_EXECUTED"]
executed_orders = executed_orders.sort_values(by='EventTime', ascending=True)

In [ ]:
# executed_orders = executed_orders.sort_values(by='EventTime', ascending=True)
# executed_orders['count'] = 1
# executed_orders['cumsum_order_qty'] = executed_orders['count'].cumsum()
# executed_orders['cumsum_qty'] = executed_orders['quantity'].cumsum()
# executed_orders.drop(columns=['count'], inplace=True)

# fig_executed_order = go.Figure()
# fig_executed_order.add_trace(go.Scatter(x=executed_orders.EventTime, y=executed_orders["cumsum_order_qty"], mode='lines', name='Exchange 0 - Executed Order Quantity'))
# fig_executed_order.add_trace(go.Scatter(x=executed_orders.EventTime, y=executed_orders["cumsum_qty"], mode='lines', name='Exchange 0 - Executed Order Volumes'))


# ex_0_submitted_orders['count'] = 1
# ex_0_submitted_orders['cumsum_order_qty'] = ex_0_submitted_orders['count'].cumsum()
# ex_0_submitted_orders['cumsum_qty'] = ex_0_submitted_orders['quantity'].cumsum()
# ex_0_submitted_orders.drop(columns=['count'], inplace=True)




# executed_order_qty = go.Figure()
# executed_order_volume = go.Figure()
# executed_order_qty.add_trace(go.Scatter(x=ex_0_submitted_orders.EventTime, y=ex_0_submitted_orders["cumsum_order_qty"], mode='lines', name='Exchange 0 - Executed Order Quantity'))
# executed_order_volume.add_trace(go.Scatter(x=ex_0_submitted_orders.EventTime, y=ex_0_submitted_orders["cumsum_qty"], mode='lines', name='Exchange 0 - Executed Order Volumes'))


# executed_order_volume

In [ ]:
"""
    Calculating the realized exchange fees for the exchange

"""

executed_orders =  logs_df[logs_df.EventType=="ORDER_EXECUTED"]
executed_orders = executed_orders[executed_orders.EventTime != 0]
executed_orders = executed_orders.sort_values(by='EventTime', ascending=True)

# create a cumsum column of order_fee

executed_orders['count'] = 1
executed_orders['cumsum_order_qty'] = executed_orders['count'].cumsum()
executed_orders['cumsum_qty'] = executed_orders['quantity'].cumsum()
executed_orders.drop(columns=['count'], inplace=True)


# divide order_fee by 100
executed_orders['order_fee'] = executed_orders['order_fee'].div(100)
executed_orders['cumsum_order_fee'] = executed_orders['order_fee'].cumsum()

fig_executed_order = go.Figure()
fig_executed_order.add_trace(go.Scatter(x=executed_orders.EventTime, y=executed_orders['cumsum_order_fee'], mode='lines', name='Exchange 0 - Market Fees Turnover'))

fig_executed_order

# Testing

In [ ]:
order_book = end_state["agents"][0].order_books["ABM"]
L1 = order_book.get_L1_snapshots()
best_bids = pd.DataFrame(L1["best_bids"],columns=["time","price","qty"])
best_asks = pd.DataFrame(L1["best_asks"],columns=["time","price","qty"])

## All times are in ns from 1970, remove the date component to put them in ns from midnight
best_bids["time"] = best_bids["time"].apply( lambda x: x - ns_date(x) )
best_asks["time"] = best_asks["time"].apply( lambda x: x - ns_date(x) )

plt.plot(best_bids.time,best_bids.price)
plt.plot(best_asks.time,best_asks.price)

band = 100
plt.ylim(100_000-band,100_000+band)

time_mesh = np.arange(
    str_to_ns("09:30:00"),
    str_to_ns("10:10:00"),
    1e9*60*10
)
_=plt.xticks(time_mesh, [ fmt_ts(time).split(" ")[1] for time in time_mesh], rotation=60 )

In [ ]:
# filter order submitted and order executed from logs
order_submitted = logs_df[logs_df.EventType.isin(["ORDER_SUBMITTED"])]
order_executed = logs_df[logs_df.EventType.isin(["ORDER_EXECUTED"])]
# sum quantity grouped by id in order_executed and create new dataframe with columns order_id quantity
order_executed_sum = order_executed.groupby(['order_id'])['quantity'].sum().reset_index()

# substract order_submitted quantity from order_executed_sum quantity and add new column to partial_left
order_submitted['partial_left'] = order_submitted['quantity'].sub(order_submitted['order_id'].map(order_executed_sum.set_index('order_id')['quantity']))

not_fully_executed = order_submitted[order_submitted.partial_left != 0]
only_fully_executed = order_submitted[order_submitted.partial_left == 0]
count_not_fully_executed = len(order_submitted[order_submitted.partial_left != 0])
# new dataframe from order_executed but only with order_id in only_full_executed and reset index 
order_executed_only_full_executed = order_executed[order_executed.order_id.isin(only_fully_executed.order_id)]

# add order_quanity from only_fully_executed to order_executed_only_full_executed as new column with "placed_quantity"
order_executed_only_full_executed['placed_quantity'] = order_executed_only_full_executed['order_id'].map(only_fully_executed.set_index('order_id')['quantity'])

# filter order_executed_only_full_executed by order_id but only take the latest time_placed order
order_executed_only_full_executed = order_executed_only_full_executed.groupby(['order_id']).tail(1).reset_index()
# time placed - time_executed and add new column to order_executed_only_full_executed
order_executed_only_full_executed['speed_of_fill'] = ((order_executed_only_full_executed['time_executed'] - order_executed_only_full_executed['time_placed']).div(1000000))

# Likelihood of a fill == fill rate Fill rate = (partial_execution / placed_quantity) * 100
order_executed_only_full_executed['fill_rate'] = (order_executed_only_full_executed['quantity'].div(order_executed_only_full_executed['placed_quantity'])).mul(100)

# average of speed_of_fill 
average_speed_of_fill = order_executed_only_full_executed['speed_of_fill'].mean()

# average of fill_rate
average_fill_rate = order_executed_only_full_executed['fill_rate'].mean()





In [ ]:
order_executed_only_full_executed["time_placed"]

In [ ]:
# speed of fills
average_speed_of_fill

In [ ]:
# fill rate (average % of orders that are filled) (partial_execution / placed_quantity) * 100
average_fill_rate

In [ ]:
order_executed_only_full_executed = order_executed_only_full_executed.sort_values(by=['time_executed']).reset_index()

fig_speed = go.Figure()
fig_speed.add_trace(go.Scatter(x=order_executed_only_full_executed.time_executed, y=order_executed_only_full_executed.speed_of_fill, mode='lines', name='Speed of executions (ms)'))

fig_speed.update_layout(title='Speed of executions', xaxis_title='Time', yaxis_title='speed in (ms)')


fig_fill_rate= go.Figure()
fig_fill_rate.add_trace(go.Scatter(x=order_executed_only_full_executed.time_executed, y=order_executed_only_full_executed.fill_rate, mode='lines', name='Fill rates of executions'))

fig_fill_rate.update_layout(title='Fill rate of executions', xaxis_title='Time', yaxis_title='% of orders filled')
fig_fill_rate

In [ ]:
execution_spreads = logs_df[logs_df.EventType.isin(["EXECUTION_SPREAD"])]
# sort by time ascending and reset index
execution_spreads = execution_spreads.sort_values(by=['time']).reset_index()


# average of realized spreads 
average_realized_spreads = execution_spreads['realized_spread'].mean()

# average of effective spreads
average_effective_spreads = execution_spreads['effective_spread'].mean()

# average of quoted spreads
average_quoted_spreads = execution_spreads['quoted_spread'].mean()

fig_spreads = go.Figure()
fig_spreads.add_trace(go.Scatter(x=execution_spreads.time, y=execution_spreads['realized_spread'], mode='lines', name='% Realized spreads'))
fig_spreads.add_trace(go.Scatter(x=execution_spreads.time, y=execution_spreads['effective_spread'], mode='lines', name='% Effective spreads'))
fig_spreads.add_trace(go.Scatter(x=execution_spreads.time, y=execution_spreads['quoted_spread'], mode='lines', name='% Quoted spreads'))
fig_spreads.update_layout(title='Spreads', xaxis_title='Time', yaxis_title='spreads')
fig_spreads
